### Импорт библиотек




In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
import seaborn as sns
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter

### Считывание данных

In [ ]:
n = ['text']
data_positive = pd.read_csv('positive.csv', sep=';', names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', names=n, usecols=['text'])

### Формирование датасета

In [ ]:
sample_size_neg = 40000
sample_size_pos = 40000
reviews_withoutshuffle = np.concatenate((data_positive['text'].values[:sample_size_pos],
                           data_negative['text'].values[:sample_size_neg]), axis=0)
labels_withoutshuffle = np.asarray([1] * sample_size_pos + [0] * sample_size_neg)

assert len(reviews_withoutshuffle) == len(labels_withoutshuffle)
from sklearn.utils import shuffle
reviews,labels = shuffle(reviews_withoutshuffle, labels_withoutshuffle, random_state=0)

### Алгоритм предварительной обработки

In [ ]:
pip install pymorphy3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 862.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.6 MB/s eta 0:00:00


In [ ]:
import nltk
import string
import pymorphy3
#from nltk.corpus import stopwords

In [ ]:
#nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#stop_words = set(stopwords.words('russian'))
morph = pymorphy3.MorphAnalyzer()

In [ ]:
rus_stopwords = {'того', 'вам', 'перед', 'можно', 'нее', 'а', 'быть', 'и', 'один', 'ему', 'будто', 'меня', 'если', 'им', 'ну', 'ли', 'никогда', 'для', 'под', 'два', 'была', 'то', 'со', 'чуть', 'вы', 'чтоб', 'от', 'нибудь', 'разве', 'опять', 'после', 'ни', 'все', 'же', 'всех', 'более', 'нельзя', 'там', 'есть', 'чего', 'моя', 'тоже', 'ведь', 'уже', 'впрочем', 'конечно', 'ж', 'них', 'нет', 'зачем', 'так', 'ее', 'тем', 'тогда', 'себе', 'их', 'чтобы', 'больше', 'даже', 'такой', 'через', 'нас', 'ней', 'вот', 'они', 'хоть', 'при', 'бы', 'совсем', 'ничего', 'мы', 'ты', 'только', 'какая', 'теперь', 'где', 'этого', 'здесь', 'наконец', 'эту', 'ей', 'свою', 'может', 'надо', 'будет', 'этот', 'всего', 'потом', 'другой', 'эти', 'у', 'тут', 'какой', 'мой', 'ним', 'она', 'хорошо', 'раз', 'него', 'по', 'куда', 'между', 'о', 'как', 'я', 'вас', 'тебя', 'были', 'вдруг', 'про', 'этом', 'к', 'потому', 'был', 'да', 'над', 'почти', 'его', 'сам', 'что', 'но', 'или', 'себя', 'из', 'до', 'тот', 'лучше', 'уж', 'три', 'когда', 'за', 'иногда', 'во', 'сейчас', 'много', 'еще', 'мне', 'чем', 'на', 'было', 'всегда', 'кто', 'в', 'об', 'он', 'с', 'всю', 'без', 'этой', 'том'}
print(rus_stopwords)

{'со', 'от', 'к', 'них', 'того', 'много', 'по', 'этого', 'там', 'какой', 'после', 'им', 'всех', 'хоть', 'меня', 'или', 'будто', 'при', 'тот', 'этот', 'из', 'впрочем', 'с', 'чтобы', 'моя', 'перед', 'тогда', 'где', 'нее', 'один', 'всего', 'чтоб', 'между', 'нет', 'во', 'всю', 'она', 'ли', 'больше', 'ж', 'еще', 'были', 'ничего', 'его', 'потому', 'нельзя', 'нас', 'про', 'здесь', 'есть', 'хорошо', 'конечно', 'уже', 'ним', 'без', 'об', 'все', 'ней', 'тоже', 'то', 'другой', 'у', 'эту', 'да', 'до', 'если', 'зачем', 'никогда', 'был', 'мой', 'за', 'иногда', 'ты', 'чуть', 'два', 'бы', 'ему', 'ни', 'ее', 'раз', 'этой', 'вы', 'себя', 'разве', 'и', 'мы', 'в', 'что', 'теперь', 'сам', 'куда', 'чего', 'тем', 'вот', 'так', 'надо', 'над', 'мне', 'можно', 'будет', 'такой', 'нибудь', 'было', 'даже', 'эти', 'какая', 'они', 'вдруг', 'быть', 'под', 'тут', 'тебя', 'него', 'как', 'чем', 'а', 'может', 'вас', 'для', 'почти', 'когда', 'себе', 'ей', 'я', 'опять', 'их', 'сейчас', 'том', 'наконец', 'свою', 'но', 'вам'

In [ ]:
def tokenize():
    punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~№'
    all_reviews = ' separator '.join(reviews)
    all_reviews = all_reviews.lower()
    all_text = ''.join([c for c in all_reviews if c not in punctuation])

    low_tokens = all_text.split()
    prep_text = []
    for token in low_tokens:
        lemma = morph.parse(token)[0].normal_form
        if lemma not in rus_stopwords:
            prep_text.append(lemma)
    all_text = ' '.join(prep_text)
    texts_split = all_text.split('separator')
    all_text = ' '.join(texts_split)
    words = all_text.split()
    return words, texts_split

words, texts_split = tokenize()

In [ ]:
new_reviews = []
for review in texts_split:
    review = review.split()
    new_text = []
    for word in review:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    new_reviews.append(new_text)

In [ ]:
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii - 1 for ii, word in enumerate(vocab, 1)}
reviews_ints = []
for review in new_reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])


In [ ]:
def add_pads(reviews_ints, seq_length):
    features = np.zeros((len(reviews_ints), seq_length), dtype=int)
    for i, row in enumerate(reviews_ints):
        if len(row) == 0:
            continue
        features[i, -len(row):] = np.array(row)[:seq_length]
    return features

### Разделение на обучающую, валидационную и тестовую выборки

In [ ]:
features = add_pads(reviews_ints, seq_length=50)
split_frac = 0.8 # 80% на обучающую выборку

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = labels[:split_idx], labels[split_idx:]
test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
valid_data = TensorDataset(torch.from_numpy(val_x), torch.from_numpy(val_y))
test_data = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))
batch_size = 50
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

### Определение режима: GPU или CPU

In [ ]:
train_gpu=torch.cuda.is_available()

print(train_gpu)

True


### Архитектура нейронной сети

In [ ]:
import torch.nn as nn

class LSTM_architecture(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, number_of_layers, drop=0.5):
        super(LSTM_architecture, self).__init__()
        self.output_size = output_size
        self.number_of_layers = number_of_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, number_of_layers,dropout=drop, batch_first=True)
        self.dropout = nn.Dropout(0.45)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, hidden_state):
        lstm_out, hidden_state = self.lstm(self.embedding(x.long()), hidden_state)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        out = self.dropout(lstm_out)
        out = self.fc(out)
        sig_out = self.sigmoid(out)
        batch_size = x.size(0)
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1]
        return sig_out, hidden_state

    def init_hidden_state(self, batch_size):
        weight = next(self.parameters()).data
        hidden_state = (weight.new(self.number_of_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.number_of_layers, batch_size, self.hidden_dim).zero_())
        return hidden_state


###Выбор гиперпараметров и инициализация сети

In [ ]:
vocab_size = len(vocab_to_int)+1
output_size = 1
embedding_dim = 128
hidden_dim = 128
n_layers = 2
model = LSTM_architecture(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
print(vocab_size)

111797


### Обучение модели

In [ ]:
test_res=[]
epochs = 3 #оптимальное количество эпох для того, чтобы модель достаточно обучилась, но не переобучилась
counter = 0
batch_num = 100
clip=5
if(train_gpu):
    model.cuda()
num_correct = 0
model.train()
for e in range(epochs):
    h = model.init_hidden_state(batch_size)
    for inputs, labels in train_loader:
        num_correct = 0
        counter += 1
        if(train_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
        h = tuple([each.data for each in h])
        model.zero_grad()
        output, h = model.forward(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        if counter % batch_num == 0:
            val_h = model.init_hidden_state(batch_size)
            val_losses = []
            model.eval()
            for inputs, labels in valid_loader:
                val_h = tuple([each.data for each in val_h])
                if(train_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.float())
                val_losses.append(val_loss.item())

                #accuracy
                pred = torch.round(output.squeeze())
                correct_tensor = pred.eq(labels.float().view_as(pred))
                correct = np.squeeze(correct_tensor.numpy()) if not train_gpu else np.squeeze(correct_tensor.cpu().numpy())
                num_correct += np.sum(correct)
                valid_acc = num_correct/len(valid_loader.dataset)

            model.train()
            print("Epoch: {} ;".format(e+1),
                  "Batch Number: {};".format(counter),
                  "Train Loss: {:.4f} ;".format(loss.item()),
                  "Valid Loss: {:.4f} ;".format(np.mean(val_losses)),
                  "Valid Accuracy: {:.4f}".format(valid_acc))


Epoch: 1 ; Batch Number: 100; Train Loss: 0.1970 ; Valid Loss: 0.3006 ; Valid Accuracy: 0.8769
Epoch: 1 ; Batch Number: 200; Train Loss: 0.2082 ; Valid Loss: 0.2851 ; Valid Accuracy: 0.8835
Epoch: 1 ; Batch Number: 300; Train Loss: 0.2193 ; Valid Loss: 0.2767 ; Valid Accuracy: 0.8846
Epoch: 1 ; Batch Number: 400; Train Loss: 0.3086 ; Valid Loss: 0.2596 ; Valid Accuracy: 0.8958
Epoch: 1 ; Batch Number: 500; Train Loss: 0.2009 ; Valid Loss: 0.2519 ; Valid Accuracy: 0.8999
Epoch: 1 ; Batch Number: 600; Train Loss: 0.3116 ; Valid Loss: 0.2579 ; Valid Accuracy: 0.9008
Epoch: 1 ; Batch Number: 700; Train Loss: 0.2416 ; Valid Loss: 0.2469 ; Valid Accuracy: 0.9002
Epoch: 1 ; Batch Number: 800; Train Loss: 0.2307 ; Valid Loss: 0.2493 ; Valid Accuracy: 0.9015
Epoch: 1 ; Batch Number: 900; Train Loss: 0.1625 ; Valid Loss: 0.2372 ; Valid Accuracy: 0.9062
Epoch: 1 ; Batch Number: 1000; Train Loss: 0.4734 ; Valid Loss: 0.2381 ; Valid Accuracy: 0.9051
Epoch: 1 ; Batch Number: 1100; Train Loss: 0.2199

### Сентимент-анализ

In [ ]:
def tokenize_text(test_review):
    test_review = test_review.lower()
    punctuation = '!"#$%&\'()*+,-./:;<=>?[\\]^_`{|}~№'
    test_text = ''.join([c for c in test_review if c not in punctuation])
    test_words = test_text.split()
    prep_test = []
    for token in test_words:
        lemma = morph.parse(token)[0].normal_form
        if lemma not in rus_stopwords:
            prep_test.append(lemma)
    new_text = []
    for word in prep_test:
        if (word[0] != '@') & ('http' not in word) & (~word.isdigit()):
            new_text.append(word)
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in new_text])

    return test_ints

In [ ]:


split_id = 64000
trained_x, remained_x = reviews[:split_id], reviews[split_id:]
trained_y, remained_y = labels[:split_id], labels[split_id:]
test_id = int(len(remained_x)*0.5)
valed_x, tested_x = remained_x[:test_id], remained_x[test_id:]
valed_y, tested_y = remained_y[:test_id], remained_y[test_id:]

In [ ]:
def predict(model, test_review, test_res, sequence_length=50):
    model.eval()
    test_ints = tokenize_text(test_review)
    seq_length=sequence_length
    features = add_pads(test_ints, seq_length)
    feature_tensor = torch.from_numpy(features)
    batch_size = feature_tensor.size(0)
    h = model.init_hidden_state(batch_size)
    if(train_gpu):
        feature_tensor = feature_tensor.cuda()
    output, h = model(feature_tensor, h)

    pred = torch.round(output.squeeze())
    print('Вероятность положительного ответа {:.6f}'.format(output.item()))

    if(pred.item()==1):
        print("Позитивное сообщение")
    else:
        print("Негативное сообщение")

In [ ]:
seq_length = 30
test_review = "хороший день сегодня"
predict(model, test_review, seq_length)


Вероятность положительного ответа 0.878721
Позитивное сообщение


In [ ]:
PATH = "model"
torch.save(model.state_dict(), PATH)